In [1]:
import os
import pandas as pd

In [2]:
%pwd

'/config/workspace/Wine-Quality-ML-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/config/workspace/Wine-Quality-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# entity
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    penalty: str
    dual: bool
    tol: float
    C: float
    target_column: str

In [6]:
from Wine_Quality_ML_Project.constants import *
from Wine_Quality_ML_Project.utils.common import read_yaml,create_directories 

In [7]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            penalty = params.penalty,
            dual = params.dual,
            tol = params.tol,
            C = params.C,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
from Wine_Quality_ML_Project import logger
import joblib
from sklearn.linear_model import LogisticRegression

In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = LogisticRegression(penalty=self.config.penalty,dual=self.config.dual,tol=self.config.tol,C=self.config.C,random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-01-19 12:25:23,718:INFO:common:yaml file: config/config.yaml loaded]
[2024-01-19 12:25:23,721:INFO:common:yaml file: params.yaml loaded]
[2024-01-19 12:25:23,725:INFO:common:yaml file: schema.yaml loaded]
[2024-01-19 12:25:23,727:INFO:common:created directory at: artifacts]
[2024-01-19 12:25:23,727:INFO:common:created directory at: artifacts/model_trainer]


/config/.conda/envs/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/config/.conda/envs/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
